In [272]:
import warnings
warnings.filterwarnings('ignore')

In [273]:
# Dependencies
import pytz
import datetime as dt
from datetime import datetime
import dateparser
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from pprint import pprint 
from scipy.stats import linregress
import scipy.stats as stats
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from suntime import Sun, SunTimeException
import os
import json
import urllib.request

# Google developer API key
from config import gkey


In [274]:
accident_data_to_load ="Resources/US_Accidents_May19.csv"
accidents_data_df = pd.read_csv(
    accident_data_to_load)

# Visualize
accidents_data_df = accidents_data_df.reset_index()
accidents_data_df.set_index('ID', inplace=True)
accidents_data_df.head()


,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,29.68,10.0,Calm,NaN,0.02,Light Rain,False,Night,Night,Night
A-2,1,OH,2,2/8/16 6:07,2/8/16 6:37,39.928059,-82.831184,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,2584.0,...,29.65,10.0,Calm,NaN,0.00,Light Rain,False,Night,Night,Day
A-3,2,OH,2,2/8/16 6:49,2/8/16 7:19,39.063148,-84.032608,0.01,Accident on OH-32 State Route 32 Westbound at ...,NaN,...,29.67,10.0,SW,3.5,NaN,Overcast,True,Night,Night,Day
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day
A-5,4,OH,2,2/8/16 7:39,2/8/16 8:09,39.627781,-84.188354,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,NaN,...,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,True,Day,Day,Day


In [275]:
#Taking only the interstates
accidents_data_df['hwy'] = accidents_data_df['Street'].str.split("-", n = 1, expand = True)[0].str.contains("I")== True
accidents_data_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,10.0,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True
A-2,1,OH,2,2/8/16 6:07,2/8/16 6:37,39.928059,-82.831184,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,2584.0,...,10.0,Calm,NaN,0.00,Light Rain,False,Night,Night,Day,False
A-3,2,OH,2,2/8/16 6:49,2/8/16 7:19,39.063148,-84.032608,0.01,Accident on OH-32 State Route 32 Westbound at ...,NaN,...,10.0,SW,3.5,NaN,Overcast,True,Night,Night,Day,False
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,9.0,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,True
A-5,4,OH,2,2/8/16 7:39,2/8/16 8:09,39.627781,-84.188354,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,NaN,...,6.0,SW,3.5,NaN,Mostly Cloudy,True,Day,Day,Day,False


In [276]:
interstate_df = accidents_data_df.loc[accidents_data_df['hwy'] == True]
interstate_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,10.0,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,9.0,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,True
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,3.0,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,True
A-27,26,OH,2,2/8/16 12:52,2/8/16 13:37,39.733219,-84.159653,0.00,Accident on Irving Ave at Wilmington Ave.,1240.0,...,7.0,West,9.2,0.00,Light Snow,True,Day,Day,Day,True
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,4.0,SW,11.5,NaN,Light Snow,False,Night,Night,Day,True


In [277]:
#Extract last letter of a string from a Pandas column to get the direction
interstate_df['Direction'] = interstate_df['Street'].str.strip().str[-1]
interstate_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True,E
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,True,S
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,True,E
A-27,26,OH,2,2/8/16 12:52,2/8/16 13:37,39.733219,-84.159653,0.00,Accident on Irving Ave at Wilmington Ave.,1240.0,...,West,9.2,0.00,Light Snow,True,Day,Day,Day,True,e
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,SW,11.5,NaN,Light Snow,False,Night,Night,Day,True,S


In [278]:
interstate_df['hwy'] = interstate_df['Direction'].str.contains("E|W|N|S")== True
interstate_df.head()


,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True,E
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,True,S
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,True,E
A-27,26,OH,2,2/8/16 12:52,2/8/16 13:37,39.733219,-84.159653,0.00,Accident on Irving Ave at Wilmington Ave.,1240.0,...,West,9.2,0.00,Light Snow,True,Day,Day,Day,False,e
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,SW,11.5,NaN,Light Snow,False,Night,Night,Day,True,S


In [279]:
hwy_df = interstate_df.loc[interstate_df['hwy'] == True]
hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True,E
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,True,S
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,True,E
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,SW,11.5,NaN,Light Snow,False,Night,Night,Day,True,S
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,WSW,9.2,0.00,Light Snow,False,Day,Day,Day,True,N


In [280]:
#converting the direction column to a list to check we get only teh valid direstions, N S E W
directionToList = hwy_df['Direction'].tolist()
print(directionToList)

['E', 'S', 'E', 'S', 'N', 'E', 'S', 'N', 'S', 'E', 'S', 'S', 'E', 'S', 'N', 'S', 'S', 'E', 'N', 'S', 'E', 'W', 'W', 'W', 'S', 'N', 'S', 'N', 'N', 'E', 'N', 'N', 'S', 'W', 'N', 'E', 'N', 'S', 'E', 'E', 'S', 'S', 'W', 'N', 'S', 'N', 'N', 'E', 'N', 'E', 'S', 'S', 'S', 'W', 'S', 'S', 'S', 'N', 'S', 'W', 'S', 'S', 'S', 'E', 'S', 'S', 'S', 'E', 'N', 'N', 'E', 'W', 'S', 'S', 'E', 'N', 'N', 'N', 'N', 'W', 'N', 'S', 'E', 'W', 'N', 'S', 'N', 'N', 'N', 'N', 'E', 'S', 'E', 'N', 'W', 'N', 'N', 'E', 'W', 'S', 'E', 'W', 'E', 'N', 'W', 'N', 'N', 'N', 'E', 'E', 'W', 'E', 'N', 'E', 'E', 'S', 'W', 'S', 'N', 'E', 'W', 'W', 'S', 'N', 'E', 'S', 'S', 'E', 'N', 'S', 'S', 'E', 'W', 'E', 'E', 'S', 'N', 'N', 'W', 'N', 'N', 'S', 'E', 'E', 'S', 'W', 'E', 'S', 'W', 'E', 'S', 'E', 'N', 'N', 'E', 'E', 'E', 'N', 'W', 'S', 'S', 'S', 'S', 'S', 'E', 'W', 'W', 'S', 'S', 'E', 'S', 'W', 'S', 'S', 'W', 'S', 'W', 'S', 'W', 'W', 'S', 'E', 'W', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'W', 'E', 'E', 'S', 'S', 'E', 'S', 'N', 'W', 'S',

In [281]:
len(hwy_df)

209060

In [282]:
#extracting the accident date
hwy_df['Accident_Date'] = hwy_df['Start_Time'].str.split(" ", n = 1, expand = True)[0]
hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction,Accident_Date
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,NaN,0.02,Light Rain,False,Night,Night,Night,True,E,2/8/16
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,True,S,2/8/16
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,4.6,0.02,Light Rain,False,Day,Day,Day,True,E,2/8/16
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,11.5,NaN,Light Snow,False,Night,Night,Day,True,S,2/9/16
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,9.2,0.00,Light Snow,False,Day,Day,Day,True,N,2/9/16


In [283]:
#Converting date from string format to date format
hwy_df['Accident_Date'] = pd.to_datetime(hwy_df['Accident_Date'])
hwy_df['Accident_Date_No_Time'] = hwy_df['Accident_Date'].dt.date
hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction,Accident_Date,Accident_Date_No_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,0.02,Light Rain,False,Night,Night,Night,True,E,2016-02-08,2016-02-08
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,NaN,Mostly Cloudy,False,Night,Day,Day,True,S,2016-02-08,2016-02-08
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,0.02,Light Rain,False,Day,Day,Day,True,E,2016-02-08,2016-02-08
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,NaN,Light Snow,False,Night,Night,Day,True,S,2016-02-09,2016-02-09
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,0.00,Light Snow,False,Day,Day,Day,True,N,2016-02-09,2016-02-09


In [284]:
hwy_df = hwy_df.drop(['hwy', 'Accident_Date'], axis=1)

In [285]:
hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date_No_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,E,2016-02-08
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,S,2016-02-08
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,E,2016-02-08
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,SW,11.5,NaN,Light Snow,False,Night,Night,Day,S,2016-02-09
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,WSW,9.2,0.00,Light Snow,False,Day,Day,Day,N,2016-02-09


In [286]:
sunset_time = []
count = 0
for lat, lng, date in zip(hwy_df.loc[:, 'Start_Lat'], hwy_df.loc[:, 'Start_Lng'],hwy_df.loc[:, 'Accident_Date_No_Time']):
    #print(lat, lng, date)
    sun = Sun(lat, lng)
    sunset_time_pst = sun.get_local_sunset_time(date)
    #print(sunset_time)
    pst_dt = dateparser.parse(str(sunset_time_pst))
    utc_dt = pst_dt.astimezone(dt.timezone.utc)
    sunset_time_utc = utc_dt.strftime('%Y-%m-%d %H:%M')
    #print(sunset_time_utc) 
    sunset_time.append(sunset_time_utc)
    count+=1
    
print(count) 

209060


In [287]:
hwy_df = hwy_df.iloc[:count, :]
hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date_No_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,E,2016-02-08
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,S,2016-02-08
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,E,2016-02-08
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,SW,11.5,NaN,Light Snow,False,Night,Night,Day,S,2016-02-09
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,WSW,9.2,0.00,Light Snow,False,Day,Day,Day,N,2016-02-09


In [288]:
hwy_df['Sunset_Time_UTC'] = sunset_time

hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date_No_Time,Sunset_Time_UTC
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,NaN,0.02,Light Rain,False,Night,Night,Night,E,2016-02-08,2016-02-08 23:04
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,S,2016-02-08,2016-02-08 23:05
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,4.6,0.02,Light Rain,False,Day,Day,Day,E,2016-02-08,2016-02-08 22:59
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,11.5,NaN,Light Snow,False,Night,Night,Day,S,2016-02-09,2016-02-09 23:06
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,9.2,0.00,Light Snow,False,Day,Day,Day,N,2016-02-09,2016-02-09 23:06


In [289]:
hwy_df = hwy_df.rename({'Accident_Date_No_Time':'Accident_Date'}, axis=1)


In [290]:
hwy_df['Accident_Time'] = hwy_df['Start_Time'].str.split(" ", n = 1, expand = True)[1]
hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date,Sunset_Time_UTC,Accident_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,0.02,Light Rain,False,Night,Night,Night,E,2016-02-08,2016-02-08 23:04,5:46
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,NaN,Mostly Cloudy,False,Night,Day,Day,S,2016-02-08,2016-02-08 23:05,7:23
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,0.02,Light Rain,False,Day,Day,Day,E,2016-02-08,2016-02-08 22:59,8:21
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,NaN,Light Snow,False,Night,Night,Day,S,2016-02-09,2016-02-09 23:06,6:41
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,0.00,Light Snow,False,Day,Day,Day,N,2016-02-09,2016-02-09 23:06,8:23


In [291]:
pytz.all_timezones

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Asmera',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti',
 'Africa/Douala',
 'Africa/El_Aaiun',
 'Africa/Freetown',
 'Africa/Gaborone',
 'Africa/Harare',
 'Africa/Johannesburg',
 'Africa/Juba',
 'Africa/Kampala',
 'Africa/Khartoum',
 'Africa/Kigali',
 'Africa/Kinshasa',
 'Africa/Lagos',
 'Africa/Libreville',
 'Africa/Lome',
 'Africa/Luanda',
 'Africa/Lubumbashi',
 'Africa/Lusaka',
 'Africa/Malabo',
 'Africa/Maputo',
 'Africa/Maseru',
 'Africa/Mbabane',
 'Africa/Mogadishu',
 'Africa/Monrovia',
 'Africa/Nairobi',
 'Africa/Ndjamena',
 'Africa/Niamey',
 'Africa/Nouakchott',
 'Africa/Ouagadougou',
 'Africa/Porto-Novo',
 'Africa/Sao_Tome',
 'Africa/Timbuktu',
 'Africa/

In [292]:
sunset_time_local = []

for date, time, timezone in zip(hwy_df.loc[:, 'Accident_Date'], hwy_df.loc[:, 'Sunset_Time_UTC'], hwy_df.loc[:, 'Timezone']):
    #print(timezone)  
    est = pytz.timezone('US/Eastern')
    pst = pytz.timezone('US/Pacific')
    mst = pytz.timezone('US/Mountain')
    cst = pytz.timezone('US/Central')
    ast = pytz.timezone('US/Alaska')
    hst = pytz.timezone('US/Hawaii')
    arst = pytz.timezone('US/Arizona')
    alst = pytz.timezone('US/Aleutian')
    eist = pytz.timezone('US/East-Indiana')
    isst = pytz.timezone('US/Indiana-Starke')
    mist = pytz.timezone('US/Michigan')
    sst = pytz.timezone('US/Samoa')
    utc = pytz.utc
    fmt = '%Y-%m-%d %H:%M'
    sunset_time_no_date = time.split(' ')[1]
    #print(sunset_time_no_date)
    utc_time = datetime(int(str(date).split('-')[0]), int(str(date).split('-')[1]), int(str(date).split('-')[2]), int(str(sunset_time_no_date).split(':')[0]), int(str(sunset_time_no_date).split(':')[1]), tzinfo=utc)
    
    #print(utc_time)
    if (timezone == 'US/Eastern'): 
        sunset_time_local.append(utc_time.astimezone(est).strftime(fmt))
    elif (timezone == 'US/Pacific'):        
        sunset_time_local.append(utc_time.astimezone(pst).strftime(fmt))
    elif (timezone == 'US/Mountain'):        
        sunset_time_local.append(utc_time.astimezone(mst).strftime(fmt))
    elif (timezone == 'US/Central'):        
        sunset_time_local.append(utc_time.astimezone(cst).strftime(fmt))
    elif (timezone == 'US/Alaska'):      
        sunset_time_local.append(utc_time.astimezone(ast).strftime(fmt))
    elif (timezone == 'US/Hawaii'):        
        sunset_time_local.append(utc_time.astimezone(hst).strftime(fmt))
    elif (timezone == 'US/Arizona'):        
        sunset_time_local.append(utc_time.astimezone(arst).strftime(fmt))
    elif (timezone == 'US/Aleutian'):        
        sunset_time_local.append(utc_time.astimezone(alst).strftime(fmt))
    elif (timezone == 'US/East-Indiana'):        
        sunset_time_local.append(utc_time.astimezone(eist).strftime(fmt))    
    elif (timezone == 'US/Indiana-Starke'):        
        sunset_time_local.append(utc_time.astimezone(isst).strftime(fmt))
    elif (timezone == 'US/Michigan'):        
        sunset_time_local.append(utc_time.astimezone(mist).strftime(fmt))
    elif (timezone == 'US/Samoa'):        
        sunset_time_local.append(utc_time.astimezone(sst).strftime(fmt))
   
    
#print(sunset_time_local)
    

In [293]:
len(sunset_time_local)

208993

In [294]:
hwy_df = hwy_df.iloc[:len(sunset_time_local), :]

In [295]:
hwy_df['Sunset_Date_Time_Local'] = sunset_time_local
hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Date_Time_Local
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,Light Rain,False,Night,Night,Night,E,2016-02-08,2016-02-08 23:04,5:46,2016-02-08 18:04
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,Mostly Cloudy,False,Night,Day,Day,S,2016-02-08,2016-02-08 23:05,7:23,2016-02-08 18:05
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,Light Rain,False,Day,Day,Day,E,2016-02-08,2016-02-08 22:59,8:21,2016-02-08 17:59
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,Light Snow,False,Night,Night,Day,S,2016-02-09,2016-02-09 23:06,6:41,2016-02-09 18:06
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,Light Snow,False,Day,Day,Day,N,2016-02-09,2016-02-09 23:06,8:23,2016-02-09 18:06


In [296]:
hwy_df['Sunset_Time_Local'] = hwy_df['Sunset_Date_Time_Local'].str.split(' ', n = 1, expand = True)[1]


In [297]:
hwy_sunset_df = hwy_df.copy()
hwy_sunset_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Date_Time_Local,Sunset_Time_Local
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,False,Night,Night,Night,E,2016-02-08,2016-02-08 23:04,5:46,2016-02-08 18:04,18:04
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,False,Night,Day,Day,S,2016-02-08,2016-02-08 23:05,7:23,2016-02-08 18:05,18:05
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,False,Day,Day,Day,E,2016-02-08,2016-02-08 22:59,8:21,2016-02-08 17:59,17:59
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,False,Night,Night,Day,S,2016-02-09,2016-02-09 23:06,6:41,2016-02-09 18:06,18:06
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,False,Day,Day,Day,N,2016-02-09,2016-02-09 23:06,8:23,2016-02-09 18:06,18:06


In [298]:
time_range_status = []
time_delta = []

def time_in_range(startTime, endTime, accidentTime):
    if startTime < endTime:
        return startTime <= accidentTime and accidentTime <= endTime
    else: #Over midnight
        return accidentTime >= startTime or accidentTime <= endTime

for accident_time, sunset_time_local,date in zip(hwy_sunset_df.loc[:, 'Accident_Time'], hwy_sunset_df.loc[:, 'Sunset_Time_Local'], hwy_sunset_df.loc[:, 'Accident_Date']):
    #print(accident_time, sunset_time_local, date)
    #print(str(int(sunset_time_local.split(':')[0]) + 1))
    start_time = str(int(sunset_time_local.split(':')[0]) - 1) + ':' + sunset_time_local.split(':')[1]
    start_time_no_seconds = datetime.strptime(start_time, '%H:%M').time()
    #print(start_time_no_seconds)
    #end_time is same as sunset local time
    end_time = str(int(sunset_time_local.split(':')[0])) + ':' + sunset_time_local.split(':')[1]
    end_time_no_seconds = datetime.strptime(end_time, '%H:%M').time()
    #print(end_time_no_seconds)
    accident_time_nosecs = accident_time.split(':')[0] + ':' + accident_time.split(':')[1]
    accident_time_no_seconds = datetime.strptime(accident_time_nosecs, '%H:%M').time()
    #print(accident_time_no_seconds)
    #print(time_in_range(start_time_no_seconds, end_time_no_seconds, accident_time_no_seconds))
    time_range_status.append(time_in_range(start_time_no_seconds, end_time_no_seconds, accident_time_no_seconds))
    if (time_in_range(start_time_no_seconds, end_time_no_seconds, accident_time_no_seconds) == True):
        #print(datetime.combine(date.min, end_time_no_seconds) - datetime.combine(date.min, accident_time_no_seconds))
        time_delta.append(datetime.combine(date.min, end_time_no_seconds) - datetime.combine(date.min, accident_time_no_seconds))
#print(time_range_status) 
#print(time_delta)

In [299]:
len(time_range_status)

208993

In [300]:
hwy_sunset_df['Accidents_Close_To_Sunset_Status'] = time_range_status
hwy_sunset_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Date_Time_Local,Sunset_Time_Local,Accidents_Close_To_Sunset_Status
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,Night,Night,Night,E,2016-02-08,2016-02-08 23:04,5:46,2016-02-08 18:04,18:04,False
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,Night,Day,Day,S,2016-02-08,2016-02-08 23:05,7:23,2016-02-08 18:05,18:05,False
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,Day,Day,Day,E,2016-02-08,2016-02-08 22:59,8:21,2016-02-08 17:59,17:59,False
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,Night,Night,Day,S,2016-02-09,2016-02-09 23:06,6:41,2016-02-09 18:06,18:06,False
A-66,65,OH,3,2/9/16 8:23,2/9/16 8:53,39.622787,-84.229858,0.01,Accident on I-75 Southbound at I-675. Expect d...,NaN,...,Day,Day,Day,N,2016-02-09,2016-02-09 23:06,8:23,2016-02-09 18:06,18:06,False


In [301]:
hwy_all_directions_sunset_df = hwy_sunset_df.loc[hwy_sunset_df['Accidents_Close_To_Sunset_Status'] == True]
hwy_all_directions_sunset_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Date_Time_Local,Sunset_Time_Local,Accidents_Close_To_Sunset_Status
ID,,,,,,,,,,,,,,,,,,,,,
A-118,117,OH,3,2/11/16 17:10,2/11/16 20:00,39.735199,-84.204948,0.01,Left lane blocked due to accident on I-75 Sout...,NaN,...,Day,Day,Day,S,2016-02-11,2016-02-11 23:08,17:10,2016-02-11 18:08,18:08,True
A-270,269,OH,3,2/18/16 17:16,2/18/16 17:46,39.953472,-82.960274,0.01,Accident on I-70 Eastbound at Exit 102 Kelton ...,NaN,...,Day,Day,Day,E,2016-02-18,2016-02-18 23:11,17:16,2016-02-18 18:11,18:11,True
A-596,595,OH,3,3/10/16 18:34,3/10/16 23:00,39.788647,-84.183907,0.01,Right lane blocked due to accident on I-75 Sou...,NaN,...,Day,Day,Day,S,2016-03-10,2016-03-10 23:39,18:34,2016-03-10 18:39,18:39,True
A-805,804,CA,3,6/21/16 19:38,6/21/16 20:08,37.692444,-122.140167,0.00,Accident on I-880 Southbound at Exit 31B Washi...,NaN,...,Day,Day,Day,N,2016-06-21,2016-06-21 03:34,19:38,2016-06-20 20:34,20:34,True
A-806,805,CA,3,6/21/16 19:34,6/21/16 20:04,37.804386,-121.989738,0.00,Accident on I-680 Northbound at Exit 36 Crow C...,NaN,...,Day,Day,Day,N,2016-06-21,2016-06-21 03:33,19:34,2016-06-20 20:33,20:33,True


In [302]:
hwy_all_directions_sunset_df['Time_Delta'] = time_delta
hwy_all_directions_sunset_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Date_Time_Local,Sunset_Time_Local,Accidents_Close_To_Sunset_Status,Time_Delta
ID,,,,,,,,,,,,,,,,,,,,,
A-118,117,OH,3,2/11/16 17:10,2/11/16 20:00,39.735199,-84.204948,0.01,Left lane blocked due to accident on I-75 Sout...,NaN,...,Day,Day,S,2016-02-11,2016-02-11 23:08,17:10,2016-02-11 18:08,18:08,True,00:58:00
A-270,269,OH,3,2/18/16 17:16,2/18/16 17:46,39.953472,-82.960274,0.01,Accident on I-70 Eastbound at Exit 102 Kelton ...,NaN,...,Day,Day,E,2016-02-18,2016-02-18 23:11,17:16,2016-02-18 18:11,18:11,True,00:55:00
A-596,595,OH,3,3/10/16 18:34,3/10/16 23:00,39.788647,-84.183907,0.01,Right lane blocked due to accident on I-75 Sou...,NaN,...,Day,Day,S,2016-03-10,2016-03-10 23:39,18:34,2016-03-10 18:39,18:39,True,00:05:00
A-805,804,CA,3,6/21/16 19:38,6/21/16 20:08,37.692444,-122.140167,0.00,Accident on I-880 Southbound at Exit 31B Washi...,NaN,...,Day,Day,N,2016-06-21,2016-06-21 03:34,19:38,2016-06-20 20:34,20:34,True,00:56:00
A-806,805,CA,3,6/21/16 19:34,6/21/16 20:04,37.804386,-121.989738,0.00,Accident on I-680 Northbound at Exit 36 Crow C...,NaN,...,Day,Day,N,2016-06-21,2016-06-21 03:33,19:34,2016-06-20 20:33,20:33,True,00:59:00


In [303]:
len(hwy_all_directions_sunset_df)

10981

In [305]:
# filtering the accidents which occurred less than 30 mins earlier than sunset
m = dt.timedelta(minutes=30)
hwy_all_directions_sunset_df["Time_Delta_Less_Than_30mins"] = (hwy_all_directions_sunset_df["Time_Delta"] < m)
hwy_all_directions_sunset_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Astronomical_Twilight,Direction,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Date_Time_Local,Sunset_Time_Local,Accidents_Close_To_Sunset_Status,Time_Delta,Time_Delta_Less_Than_30mins
ID,,,,,,,,,,,,,,,,,,,,,
A-118,117,OH,3,2/11/16 17:10,2/11/16 20:00,39.735199,-84.204948,0.01,Left lane blocked due to accident on I-75 Sout...,NaN,...,Day,S,2016-02-11,2016-02-11 23:08,17:10,2016-02-11 18:08,18:08,True,00:58:00,False
A-270,269,OH,3,2/18/16 17:16,2/18/16 17:46,39.953472,-82.960274,0.01,Accident on I-70 Eastbound at Exit 102 Kelton ...,NaN,...,Day,E,2016-02-18,2016-02-18 23:11,17:16,2016-02-18 18:11,18:11,True,00:55:00,False
A-596,595,OH,3,3/10/16 18:34,3/10/16 23:00,39.788647,-84.183907,0.01,Right lane blocked due to accident on I-75 Sou...,NaN,...,Day,S,2016-03-10,2016-03-10 23:39,18:34,2016-03-10 18:39,18:39,True,00:05:00,True
A-805,804,CA,3,6/21/16 19:38,6/21/16 20:08,37.692444,-122.140167,0.00,Accident on I-880 Southbound at Exit 31B Washi...,NaN,...,Day,N,2016-06-21,2016-06-21 03:34,19:38,2016-06-20 20:34,20:34,True,00:56:00,False
A-806,805,CA,3,6/21/16 19:34,6/21/16 20:04,37.804386,-121.989738,0.00,Accident on I-680 Northbound at Exit 36 Crow C...,NaN,...,Day,N,2016-06-21,2016-06-21 03:33,19:34,2016-06-20 20:33,20:33,True,00:59:00,False


In [308]:
hwy_all_directions_sunset_less_than_30mins_df = hwy_all_directions_sunset_df.loc[hwy_all_directions_sunset_df['Time_Delta_Less_Than_30mins'] == True]
hwy_all_directions_sunset_less_than_30mins_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Astronomical_Twilight,Direction,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Date_Time_Local,Sunset_Time_Local,Accidents_Close_To_Sunset_Status,Time_Delta,Time_Delta_Less_Than_30mins
ID,,,,,,,,,,,,,,,,,,,,,
A-596,595,OH,3,3/10/16 18:34,3/10/16 23:00,39.788647,-84.183907,0.01,Right lane blocked due to accident on I-75 Sou...,NaN,...,Day,S,2016-03-10,2016-03-10 23:39,18:34,2016-03-10 18:39,18:39,True,00:05:00,True
A-811,810,CA,3,6/21/16 20:03,6/21/16 20:48,37.766987,-121.335495,0.00,Two lanes blocked due to accident on I-205 Wes...,NaN,...,Day,W,2016-06-21,2016-06-21 03:31,20:03,2016-06-20 20:31,20:31,True,00:28:00,True
A-812,811,CA,3,6/21/16 20:19,6/21/16 20:49,37.700043,-121.779655,0.00,Accident on I-580 Eastbound at Exit 51 Isabel ...,NaN,...,Day,E,2016-06-21,2016-06-21 03:32,20:19,2016-06-20 20:32,20:32,True,00:13:00,True
A-1177,1176,CA,3,6/24/16 20:07,6/24/16 21:07,38.710537,-121.306374,0.00,Accident on I-80 Westbound at Exit 100 Antelop...,NaN,...,Day,W,2016-06-24,2016-06-24 03:34,20:07,2016-06-23 20:34,20:34,True,00:27:00,True
A-1178,1177,CA,3,6/24/16 20:22,6/24/16 20:52,37.861172,-122.032990,0.00,Accident on I-680 Northbound at Exit 43 Livorn...,NaN,...,Day,N,2016-06-24,2016-06-24 03:34,20:22,2016-06-23 20:34,20:34,True,00:12:00,True


In [311]:
print(len(hwy_all_directions_sunset_less_than_30mins_df))
directionToList = hwy_all_directions_sunset_less_than_30mins_df['Direction'].tolist()
#print(directionToList)

5266


In [323]:
time_diff_min = []
for time_delta in zip(hwy_all_directions_sunset_less_than_30mins_df.loc[:, 'Time_Delta']):
    mins = int(str(time_delta).split(":")[1])
    #print(mins)
    time_diff_min.append(mins)

In [324]:
hwy_all_directions_sunset_less_than_30mins_df['Time_Diff_Mins'] = time_diff_min
hwy_all_directions_sunset_less_than_30mins_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Direction,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Date_Time_Local,Sunset_Time_Local,Accidents_Close_To_Sunset_Status,Time_Delta,Time_Delta_Less_Than_30mins,Time_Diff_Mins
ID,,,,,,,,,,,,,,,,,,,,,
A-596,595,OH,3,3/10/16 18:34,3/10/16 23:00,39.788647,-84.183907,0.01,Right lane blocked due to accident on I-75 Sou...,NaN,...,S,2016-03-10,2016-03-10 23:39,18:34,2016-03-10 18:39,18:39,True,00:05:00,True,5
A-811,810,CA,3,6/21/16 20:03,6/21/16 20:48,37.766987,-121.335495,0.00,Two lanes blocked due to accident on I-205 Wes...,NaN,...,W,2016-06-21,2016-06-21 03:31,20:03,2016-06-20 20:31,20:31,True,00:28:00,True,28
A-812,811,CA,3,6/21/16 20:19,6/21/16 20:49,37.700043,-121.779655,0.00,Accident on I-580 Eastbound at Exit 51 Isabel ...,NaN,...,E,2016-06-21,2016-06-21 03:32,20:19,2016-06-20 20:32,20:32,True,00:13:00,True,13
A-1177,1176,CA,3,6/24/16 20:07,6/24/16 21:07,38.710537,-121.306374,0.00,Accident on I-80 Westbound at Exit 100 Antelop...,NaN,...,W,2016-06-24,2016-06-24 03:34,20:07,2016-06-23 20:34,20:34,True,00:27:00,True,27
A-1178,1177,CA,3,6/24/16 20:22,6/24/16 20:52,37.861172,-122.032990,0.00,Accident on I-680 Northbound at Exit 43 Livorn...,NaN,...,N,2016-06-24,2016-06-24 03:34,20:22,2016-06-23 20:34,20:34,True,00:12:00,True,12


In [325]:
# Extract individual groups
group1 = hwy_all_directions_sunset_less_than_30mins_df[hwy_all_directions_sunset_less_than_30mins_df["Direction"] == "E"]["Time_Diff_Mins"]
group2 = hwy_all_directions_sunset_less_than_30mins_df[hwy_all_directions_sunset_less_than_30mins_df["Direction"] == "N"]["Time_Diff_Mins"]
group3 = hwy_all_directions_sunset_less_than_30mins_df[hwy_all_directions_sunset_less_than_30mins_df["Direction"] == "S"]["Time_Diff_Mins"]
group4 = hwy_all_directions_sunset_less_than_30mins_df[hwy_all_directions_sunset_less_than_30mins_df["Direction"] == "W"]["Time_Diff_Mins"]

In [326]:
# Perform the ANOVA
stats.f_oneway(group1, group2, group3, group4)

F_onewayResult(statistic=0.22213286793966605, pvalue=0.8810724774699882)